In [2]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
from LSTM_Attention import SimpleModel as Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv

from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import itertools
import argparse
import os,sys,re
import numpy as np 
from Bio import SeqIO
import pickle as pkl
import scipy.io as sio
%matplotlib inline

In [3]:
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

In [4]:
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

class EmbeddingSeq(nn.Module):
    def __init__(self,weight_dict_path):
        """
        Inputs:
            weight_dict_path: path of pre-trained embeddings of RNA/dictionary
        """
        super(EmbeddingSeq,self).__init__()
        weight_dict = pickle.load(open(weight_dict_path,'rb'))

        weights = torch.FloatTensor(list(weight_dict.values())).cuda()
        num_embeddings = len(list(weight_dict.keys()))
        embedding_dim = 300

        self.embedding = nn.Embedding(num_embeddings=num_embeddings,embedding_dim=embedding_dim)
        self.embedding.weight = nn.Parameter(weights)
        self.embedding.weight.requires_grad = False

    def forward(self,x):

        out = self.embedding(x.type(torch.cuda.LongTensor))

        return out

def seq2index(seqs,my_dict,window=3,save_data=False):
    """
    Convert single RNA sequences to k-mers representation.
        Inputs: ['ACAUG','CAACC',...] of equal length RNA seqs
        Example: 'ACAUG' ----> [ACA,CAU,AUG] ---->[21,34,31]
    """

    num_samples = len(seqs)
    temp = []
    for k in range(num_samples):
        length = len(seqs[k])
        seqs_kmers = [seqs[k][i:i+window] for i in range(0,length-window+1)]
        temp.append(seqs_kmers)


    seq_kmers = pd.DataFrame(data = np.concatenate(temp,axis=0))

    # load pretained word2vec embeddings

    word2index = word2index_(my_dict)

    seq_kmers_index = seq_kmers.applymap(lambda x: mapfun(x,my_dict))


    return seq_kmers_index.to_numpy()
def word2index_(my_dict):
    word2index = dict()
    for index, ele in enumerate(list(my_dict.keys())):
        word2index[ele] = index

    return word2index

def mapfun(x,my_dict):
    if x not in list(my_dict.keys()):
        return None
    else:
        return word2index_(my_dict)[x]
    
embed = EmbeddingSeq('features/embeddings_12RM.pkl')
embeddings_dict = pickle.load(open('features/embeddings_12RM.pkl','rb'))

train_seq = fa_seq("data/Mouse/mouse_train.fasta")
trainLabel = np.append(np.ones(int(len(train_seq)/2)),np.zeros(int(len(train_seq)-len(train_seq)/2)),axis=0)
seq_T = []
for i in train_seq:
    seq_T.append(i.replace('U','T'))
x = seq2index(seq_T,embeddings_dict)
seqs_kmers_index = torch.transpose(torch.from_numpy(x),0,1)
trainData = np.array(embed(seqs_kmers_index).view(len(train_seq),-1,300).cpu())

test_seq = fa_seq("data/Mouse/mouse_indep.fasta")
testLabel = np.append(np.ones(int(len(test_seq)/2)),np.zeros(int(len(test_seq)-len(test_seq)/2)),axis=0)
seq_T = []
for i in test_seq:
    seq_T.append(i.replace('U','T'))
x = seq2index(seq_T,embeddings_dict)
seqs_kmers_index = torch.transpose(torch.from_numpy(x),0,1)
testData = np.array(embed(seqs_kmers_index).view(len(test_seq),-1,300).cpu())

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [5]:
#CNN
class Model_CNN(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1,3,kernel_size=5,stride=2,padding=2)
        self.conv2 = nn.Conv2d(3,3,kernel_size=5,stride=2,padding=2)
        self.conv3 = nn.Conv2d(3,3,kernel_size=5,stride=2,padding=2)
        
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        k1 = int((k1+2*2-5)/2)+1
        k2 = int((k2+2*2-5)/2)+1
        k1 = int((k1+2*2-5)/2)+1
        k2 = int((k2+2*2-5)/2)+1
        
        self.fn1 = nn.Linear(3*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        
        x1 = x.contiguous().view(-1,1,x.shape[1],x.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = self.conv2(x1)
        x1 = self.relu(x1)
        x1 = self.conv3(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

#LSTM
class Model_LSTM(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_LSTM, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=False)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        #self.fn1 = nn.Linear(dim1*dim2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = self.relu(x1)
        x1 = self.bm1(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

#BiLSTM
class Model_BiLSTM(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_BiLSTM, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=True)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = x1[:,:,0:300]+x1[:,:,300:600]
        x1 = self.relu(x1)
        x1 = self.bm1(x1)
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out
    
#Attention
class Model_Attention(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_Attention, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.self_A = SelfAttention(dim2)
        self.self_B = SelfAttention(dim1)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        if x1.shape[1] == 1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
            x1,attn = self.self_B(x1,x1,x1)
        else:
            x1,attn = self.self_A(x1,x1,x1)
        if x1.shape[2]==1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
        x1 = self.bm1(x1)
        x1 = self.relu(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

#LSTM_Attention_CNN
class Model_LSTM_Attention_CNN(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_LSTM_Attention_CNN, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.self_A = SelfAttention(dim2)
        self.self_B = SelfAttention(dim1)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=False)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)
        #self.encoder = nn.Sequential()

    def forward(self, x):
        
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = self.relu(x1)
        if x1.shape[1] == 1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
            x1,attn = self.self_B(x1,x1,x1)
        else:
            x1,attn = self.self_A(x1,x1,x1)
        if x1.shape[2]==1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
        x1 = self.bm1(x1)
        x1 = self.relu(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out
    
#BiLSTM_Attention_CNN
class Model_BiLSTM_Attention_CNN(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_BiLSTM_Attention_CNN, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.self_A = SelfAttention(dim2)
        self.self_B = SelfAttention(dim1)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=True)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = x1[:,:,0:300]+x1[:,:,300:600]
        x1 = self.relu(x1)
        if x1.shape[1] == 1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
            x1,attn = self.self_B(x1,x1,x1)
        else:
            x1,attn = self.self_A(x1,x1,x1)
        if x1.shape[2]==1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
        x1 = self.bm1(x1)
        x1 = self.relu(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

In [6]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

def attention(query, key, value, mask=None, dropout=None):  # q,k,v: [batch, h, seq_len, d_k]
    d_k = query.size(-1)  # query的维度
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)  # 打分机制 [batch, h, seq_len, seq_len]
    p_atten = F.softmax(scores, dim=-1)  # 对最后一个维度归一化得分, [batch, h, seq_len, seq_len]
    if dropout is not None:
        p_atten = dropout(p_atten)
    return torch.matmul(p_atten, value), p_atten  # [batch, h, seq_len, d_k] 作矩阵的乘法

class PositionalEncoding(nn.Module):

    def __init__(self, dim1, dim2, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()

        #if dim % 2 != 0:
        #    raise ValueError("Cannot use sin/cos positional encoding with "
        #                     "odd dim (got dim={:d})".format(dim))

        """
        构建位置编码pe
        pe公式为：
        PE(pos,2i/2i+1) = sin/cos(pos/10000^{2i/d_{model}})
        """
        pe = torch.zeros(max_len, dim2)  # max_len 是解码器生成句子的最长的长度，假设是 10
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term0 = torch.exp((torch.arange(0, dim2, 2, dtype=torch.float) * -(math.log(10000.0) / dim2)))
        div_term1 = torch.exp((torch.arange(1, dim2, 2, dtype=torch.float) * -(math.log(10000.0) / dim2)))
        
        pe[:, 0::2] = torch.sin(position.float() * div_term0)
        pe[:, 1::2] = torch.cos(position.float() * div_term1)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        #self.drop_out = nn.Dropout(p=dropout)
        self.dim2 = dim2
        self.bm1 = nn.BatchNorm1d(dim1,eps=1e-05)

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim2)
        if step is None:
            emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
        else:
            emb = emb + self.pe[step]
        #emb = self.drop_out(emb)
        emb = self.bm1(emb.to(torch.float32))
        return emb
    
def clones(module, N):  #定义clones方法
    return nn.ModuleList([copy.deepcopy(module)
                          for _ in range(N)])  #让原来变量不影响,且克隆module N次

class SelfAttention(nn.Module):  #多头注意力机制

    def __init__(self,embedding_dim, dropout=0.1):
        super(SelfAttention, self).__init__()
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)  #克隆四份Linear网络层
        self.dropout = nn.Dropout(p=dropout)  #定义Dropout层

    def forward(self,query,key,value,mask=None):  # q,k,v: [batch, seq_len, embedding_dim]
        nbatches = query.shape[0]  #批数量
        query, key, value = [
            l(x) for l, x in zip(self.linears,
                            (query.to(torch.float32),
                             key.to(torch.float32),
                             value.to(torch.float32)))
        ]  #获取zip的query,key,value权重矩阵
        attn, p_atten = attention(query,key,value,mask=mask,dropout=self.dropout)
        out = self.linears[-1](attn)  #得到最后一层线性层的输出
        return out,p_atten  #返回out结果

In [7]:
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import copy
from torch.autograd import Variable
from sklearn.model_selection import KFold

def train(model,data,label,epoch,train_device,model_dir,batch_size):
    if os.path.exists(model_dir+'model.pt'):
        model_train = torch.load(model_dir+'/model.pt')
    else:
        model_train = model
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model_train.parameters(),lr=0.003)  #改变学习率
    dataX = torch.Tensor(data).clone().detach()
    label = torch.Tensor(label).clone().detach()
    train_data = TensorDataset(dataX, label)
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    running_loss = 0.0
    model_train = model_train.to(train_device)
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        #inputs = inputs.reshape(inputs.shape[0], 1, inputs.shape[1])
        inputs = inputs.to(train_device)
        target = target.to(train_device)
        target = target.reshape(target.shape[0],1)
        optimizer.zero_grad()
        outputs = model_train(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        #print(type(loss))
        running_loss += loss.item()
        if batch_idx == len(dataX)//batch_size:
            #print('[%d, %5d] epoch loss: %.3f' %(epoch+1,batch_idx+1,running_loss))
            print(running_loss)
    save_model(model_train,model_dir)
    model_train = torch.load(model_dir+'/model.pt')
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    th,_,_,_,_,_,_,_,_,_ = metricsCal.evaluate(model_train,train_loader,train_device)
    return running_loss,th

def test(data,label,best_auc,test_device,model_dir,batch_size,th):
    model_test = load_model(model_dir)
    #model_test.eval()
    #model_test.to(test_device)
    data = torch.Tensor(data).clone().detach()#torch.Tensor(data)
    label = torch.Tensor(label).clone().detach()#.requires_grad_(True)torch.Tensor(label)
    test_data = TensorDataset(data,label)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    _,_,_,_,Sen,Spe, Acc, mcc, AUC = metricsCal.evaluate(model_test,test_loader,test_device,False,th)
    
    print('Accuracy on test set: %d %%' %Acc)
    print('Sensitivity on test set: %d %%' %Sen)
    print('Speciality on test set: %d %%' %Spe)
    print('MCC on test set: %.3f' %mcc)
    print('auc on test set: %.3f' %AUC)
    if(AUC > best_auc):
        torch.save(model_test,model_dir+'model_best.pt')
    return Sen,Spe,Acc, mcc, AUC

def independTest(data,label,test_device,model_dir,batch_size,th):
    model_test = load_bestModel(model_dir)
    data = torch.Tensor(data).clone().detach()#torch.Tensor(data)
    label = torch.Tensor(label).clone().detach()#.requires_grad_(True)torch.Tensor(label)
    test_data = TensorDataset(data,label)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    _,_,_,_,Sen,Spe, Acc, mcc, AUC = metricsCal.evaluate(model_test,test_loader,test_device,False,th)
    print(Acc,mcc,AUC)
    print('Accuracy on test set: %d %%' %Acc)
    print('Sensitivity on test set: %d %%' %Sen)
    print('Speciality on test set: %d %%' %Spe)
    print('MCC on test set: %.3f' %mcc)
    print('auc on test set: %.3f' %AUC)
    return Acc, mcc, AUC

def load_model(model_dir):
    if os.path.exists(model_dir+'model.pt'):
        model_load = torch.load(model_dir+'model.pt')
    return model_load

def save_model(model_save,model_dir):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    torch.save(model_save, model_dir+'model.pt')

def load_bestModel(model_dir):
    if os.path.exists(model_dir+'model_best.pt'):
        model_load = torch.load(model_dir+'model_best.pt')#,map_location='cuda:0')
    return model_load

In [8]:
def independResult(data,label,test_device,model_dir,batch_size,th=0.5):
    model_test = load_bestModel(model_dir)
    data = torch.Tensor(data).clone().detach()#torch.Tensor(data)
    label = torch.Tensor(label).clone().detach()#.requires_grad_(True)torch.Tensor(label)
    test_data = TensorDataset(data,label)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    
    return evaluate_result(model_test,test_loader,test_device,False)

def evaluate_result(model, dataloader, device, is_train=True, threshold=0.5):
    model.eval()
    y_true = torch.tensor([],dtype=torch.int)
    y_score = torch.tensor([])
    #for data in tqdm(dataloader):
    model = model.to(device)
    for data in dataloader:
        #if not isinstance(model, Res_Net):
        if 1==1:
            inputs,y = data
            inputs = inputs.to(device)
            out = model(inputs)
        out = out.squeeze(dim=-1)
        #out = torch.sigmoid(out)
        y_true = torch.cat((y_true, y.int().detach().cpu()))
        y_score = torch.cat((y_score, out.detach().cpu()))  #detach去除梯度，然后cpu()，然后cat将其连接起来
    y_true = y_true.numpy()
    y_score = y_score.numpy()
    return y_score, y_true

In [9]:
X_All = []
Label = []

In [10]:
trainData_w2v = trainData
testData_w2v = testData

In [11]:
import os
import math
from sklearn.metrics import roc_auc_score
#先做交叉验证，看看多少个epoch合适
rows = 10
row = -1
max_epochs = 3
max_patience = 25
batch_size = 256

X_BiLSTM = []
test_BiLSTM = []

#然后全部all_in 一个模型出来
#%env CUDA_LAUNCH_BLOCKING=1
kf = KFold(10,True,0)
x = 0
#第三重循环
for i,[train_index, test_index] in enumerate(kf.split(trainData)):

    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model_dir = "Model/Model_BiLSTM_w2v/KFold_" + str(i)+"/"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'

    y_pred,y_true = independResult(X_test,Y_test,device,model_dir,batch_size)
    X_BiLSTM.append(y_pred)
    
    test_pred,_ = independResult(testData,testLabel,device,model_dir,batch_size)
    test_BiLSTM.append(test_pred)
    Label.append(Y_test)
    x += roc_auc_score(Y_test,y_pred)

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: To copy construct

In [12]:
trainData = trainData_w2v.reshape(9126,-1)
testData = testData_w2v.reshape(2000,-1)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost import plot_importance
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier

kf = KFold(10,True,0)
X_XGBoost = []
test_XGBoost = []
for i,[train_index, test_index] in enumerate(kf.split(trainData)):

    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    clf_xgb = XGBClassifier(eval_metric=['logloss','auc','error'],max_depth = 12,subsample=0.6356535940131729,
              n_estimators= 391,gamma=2.4536762630409314,learning_rate= 0.03329907042228497,colsample_bytree=0.7668923593638199)
        
    
    clf_xgb.fit(X_train,Y_train)

    X_XGBoost.append(   np.column_stack((clf_xgb.predict_proba(X_test)[:,1])))
               # clf_et.predict_proba(X_test)[:,1]))
               # clf_lgbm.predict_proba(X_test)[:,1],
               # clf_et.predict_proba(X_test)[:,1],
               # clf_gb.predict_proba(X_test)[:,1],
               #clf_svm.decision_function(X_test)))
    
    test_XGBoost.append(   np.column_stack((clf_xgb.predict_proba(testData)[:,1])))
               # clf_et.predict_proba(testData)[:,1]))
               # clf_lgbm.predict_proba(testData)[:,1],
               # clf_et.predict_proba(testData)[:,1],
               # clf_gb.predict_proba(testData)[:,1],
               #clf_svm.decision_function(testData)))
    
    #Label.append(Y_test)
print("success")

success


In [18]:
from thundersvm import SVC

kf = KFold(10,True,0)
X_SVM = []
test_SVM = []
for i,[train_index, test_index] in enumerate(kf.split(trainData)):

    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
#     clf = CatBoostClassifier(learning_rate=0.06318730785749581,depth=8,
#                                  iterations=412,l2_leaf_reg=2.272774770449704,task_type="GPU",early_stopping_rounds=100,verbose=0)
    clf = SVC(C=8.400207912920449,gamma=0.0009752187807030065)
    
    clf.fit(X_train,Y_train)
    #clf.

    X_SVM.append(  np.column_stack(#clf.predict_proba(X_test)[:,1])))
               # clf_et.predict_proba(X_test)[:,1]))
               # clf_lgbm.predict_proba(X_test)[:,1],
               # clf_et.predict_proba(X_test)[:,1],
               # clf_gb.predict_proba(X_test)[:,1],
               clf.decision_function(X_test)))
    
    test_SVM.append(   np.column_stack(#clf.predict_proba(testData)[:,1])))
               # clf_et.predict_proba(testData)[:,1]))
               # clf_lgbm.predict_proba(testData)[:,1],
               # clf_et.predict_proba(testData)[:,1],
               # clf_gb.predict_proba(testData)[:,1],
               clf.decision_function(testData)))
    
    #Label.append(Y_test)
print("success")

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


success


In [19]:
from lightgbm import LGBMClassifier

kf = KFold(10,True,0)
X_LGBM = []
test_LGBM = []
for i,[train_index, test_index] in enumerate(kf.split(trainData)):

    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    clf = LGBMClassifier(learning_rate=0.03707304475592868,max_depth=5,n_estimators=246,
                             min_split_gain=0.19952007375316028,subsample=0.9062373540227182,
                             colsample_bytree=0.8008894112874834,lambda_l1=1.8373204960674205,
                             num_leaves=38,verbose=0)
        
    
    clf.fit(X_train,Y_train)

    X_LGBM.append(  np.column_stack((clf.predict_proba(X_test)[:,1])))
               # clf_et.predict_proba(X_test)[:,1]))
               # clf_lgbm.predict_proba(X_test)[:,1],
               # clf_et.predict_proba(X_test)[:,1],
               # clf_gb.predict_proba(X_test)[:,1],
               #clf_svm.decision_function(X_test)))
    
    test_LGBM.append(   np.column_stack((clf.predict_proba(testData)[:,1])))
               # clf_et.predict_proba(testData)[:,1]))
               # clf_lgbm.predict_proba(testData)[:,1],
               # clf_et.predict_proba(testData)[:,1],
               # clf_gb.predict_proba(testData)[:,1],
               #clf_svm.decision_function(testData)))
    
    #Label.append(Y_test)
print("success")

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


[LightGBM] [Warning] lambda_l1 is set=1.8373204960674205, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8373204960674205
[LightGBM] [Warning] lambda_l1 is set=1.8373204960674205, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8373204960674205
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.691247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] lambda_l1 is set=1.8373204960674205, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8373204960674205
[LightGBM] [Warning] lambda_l1 is set=1.8373204960674205, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8373204960674205
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.667390 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [20]:
from sklearn.ensemble import AdaBoostClassifier

kf = KFold(10,True,0)
X_AdaBoost = []
test_AdaBoost = []
for i,[train_index, test_index] in enumerate(kf.split(trainData)):

    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    clf = AdaBoostClassifier(n_estimators=558,learning_rate=0.09731666158675417)
        
    
    clf.fit(X_train,Y_train)

    X_AdaBoost.append(  np.column_stack((clf.predict_proba(X_test)[:,1])))
               # clf_et.predict_proba(X_test)[:,1]))
               # clf_lgbm.predict_proba(X_test)[:,1],
               # clf_et.predict_proba(X_test)[:,1],
               # clf_gb.predict_proba(X_test)[:,1],
               #clf_svm.decision_function(X_test)))
    
    test_AdaBoost.append(   np.column_stack((clf.predict_proba(testData)[:,1])))
               # clf_et.predict_proba(testData)[:,1]))
               # clf_lgbm.predict_proba(testData)[:,1],
               # clf_et.predict_proba(testData)[:,1],
               # clf_gb.predict_proba(testData)[:,1],
               #clf_svm.decision_function(testData)))
    
    #Label.append(Y_test)
print("success")

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


success


In [21]:
from sklearn.ensemble import ExtraTreesClassifier

kf = KFold(10,True,0)
X_ExtraTree = []
test_ExtraTree = []
for i,[train_index, test_index] in enumerate(kf.split(trainData)):

    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    clf = ExtraTreesClassifier(n_estimators=1164 ,
                                   min_samples_split=4,max_depth=40)
        
    
    clf.fit(X_train,Y_train)

    X_ExtraTree.append(  np.column_stack((clf.predict_proba(X_test)[:,1])))
               # clf_et.predict_proba(X_test)[:,1]))
               # clf_lgbm.predict_proba(X_test)[:,1],
               # clf_et.predict_proba(X_test)[:,1],
               # clf_gb.predict_proba(X_test)[:,1],
               #clf_svm.decision_function(X_test)))
    
    test_ExtraTree.append(   np.column_stack((clf.predict_proba(testData)[:,1])))
               # clf_et.predict_proba(testData)[:,1]))
               # clf_lgbm.predict_proba(testData)[:,1],
               # clf_et.predict_proba(testData)[:,1],
               # clf_gb.predict_proba(testData)[:,1],
               #clf_svm.decision_function(testData)))
    
    Label.append(Y_test)
print("success")

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


success


In [22]:
len(Label)

10

In [23]:
Y_kk = np.hstack((Label[0],Label[1],Label[2],Label[3],Label[4],Label[5],Label[6],Label[7],Label[8],Label[9]))

valData_BiLSTM = np.hstack((X_BiLSTM[0],X_BiLSTM[1],X_BiLSTM[2],X_BiLSTM[3],X_BiLSTM[4],X_BiLSTM[5],X_BiLSTM[6],X_BiLSTM[7],X_BiLSTM[8],X_BiLSTM[9]))
test_BiLSTM = np.vstack((test_BiLSTM[0],test_BiLSTM[1],test_BiLSTM[2],test_BiLSTM[3],test_BiLSTM[4],test_BiLSTM[5],test_BiLSTM[6],test_BiLSTM[7],test_BiLSTM[8],test_BiLSTM[9]))

valData_XGBoost = np.hstack((X_XGBoost[0],X_XGBoost[1],X_XGBoost[2],X_XGBoost[3],X_XGBoost[4],X_XGBoost[5],X_XGBoost[6],X_XGBoost[7],X_XGBoost[8],X_XGBoost[9]))
test_XGBoost = np.vstack((test_XGBoost[0],test_XGBoost[1],test_XGBoost[2],test_XGBoost[3],test_XGBoost[4],test_XGBoost[5],test_XGBoost[6],test_XGBoost[7],test_XGBoost[8],test_XGBoost[9]))

valData_SVM = np.hstack((X_SVM[0],X_SVM[1],X_SVM[2],X_SVM[3],X_SVM[4],X_SVM[5],X_SVM[6],X_SVM[7],X_SVM[8],X_SVM[9]))
test_SVM = np.vstack((test_SVM[0],test_SVM[1],test_SVM[2],test_SVM[3],test_SVM[4],test_SVM[5],test_SVM[6],test_SVM[7],test_SVM[8],test_SVM[9]))

valData_LGBM = np.hstack((X_LGBM[0],X_LGBM[1],X_LGBM[2],X_LGBM[3],X_LGBM[4],X_LGBM[5],X_LGBM[6],X_LGBM[7],X_LGBM[8],X_LGBM[9]))
test_LGBM = np.vstack((test_LGBM[0],test_LGBM[1],test_LGBM[2],test_LGBM[3],test_LGBM[4],test_LGBM[5],test_LGBM[6],test_LGBM[7],test_LGBM[8],test_LGBM[9]))

valData_AdaBoost = np.hstack((X_AdaBoost[0],X_AdaBoost[1],X_AdaBoost[2],X_AdaBoost[3],X_AdaBoost[4],X_AdaBoost[5],X_AdaBoost[6],X_AdaBoost[7],X_AdaBoost[8],X_AdaBoost[9]))
test_AdaBoost = np.vstack((test_AdaBoost[0],test_AdaBoost[1],test_AdaBoost[2],test_AdaBoost[3],test_AdaBoost[4],test_AdaBoost[5],test_AdaBoost[6],test_AdaBoost[7],test_AdaBoost[8],test_AdaBoost[9]))

valData_ExtraTree = np.hstack((X_ExtraTree[0],X_ExtraTree[1],X_ExtraTree[2],X_ExtraTree[3],X_ExtraTree[4],X_ExtraTree[5],X_ExtraTree[6],X_ExtraTree[7],X_ExtraTree[8],X_ExtraTree[9]))
test_ExtraTree = np.vstack((test_ExtraTree[0],test_ExtraTree[1],test_ExtraTree[2],test_ExtraTree[3],test_ExtraTree[4],test_ExtraTree[5],test_ExtraTree[6],test_ExtraTree[7],test_ExtraTree[8],test_ExtraTree[9]))


In [24]:
pd.DataFrame(np.vstack((valData_BiLSTM,valData_XGBoost,valData_SVM,valData_LGBM,valData_AdaBoost,valData_ExtraTree,Y_kk)).T).to_csv("Result/w2v_result_val.csv",header=['BiLSTM', 'XGBoost', 'SVM', 'LightGBM', 'AdaBoost', 'ExtraTree','True'])

In [25]:
kl = ['BiLSTM', 'XGBoost', 'SVM', 'LightGBM', 'AdaBoost', 'ExtraTree']
ml = []
for i in range(len(kl)):
    for j in range(10):
        ml.append(str(kl[i])+"_KFold_"+str(j) )
ml.append("True")

In [26]:
pd.DataFrame(np.vstack((test_BiLSTM,test_XGBoost,test_SVM,test_LGBM,test_AdaBoost,test_ExtraTree,testLabel)).T).to_csv("Result/w2v_result_test.csv",header=ml)

In [28]:
pd.read_csv("Result/w2v_result_val.csv")

,Unnamed: 0,BiLSTM,XGBoost,SVM,LightGBM,AdaBoost,ExtraTree,True
0,0,0.282574,0.356637,-0.102971,0.266270,0.496027,0.465922,1.0
1,1,0.860681,0.892340,0.296087,0.915566,0.504056,0.791953,1.0
2,2,0.624915,0.715984,0.543875,0.787594,0.503708,0.577606,1.0
3,3,0.297757,0.402987,0.247881,0.407767,0.498038,0.630298,1.0
4,4,0.693888,0.803717,0.462078,0.838077,0.505664,0.724084,1.0
...,...,...,...,...,...,...,...,...
9121,9121,0.496472,0.592297,-0.186698,0.587815,0.499640,0.567440,0.0
9122,9122,0.373274,0.110602,-0.154241,0.277313,0.494551,0.411942,0.0
9123,9123,0.387534,0.108579,-0.088065,0.148655,0.494654,0.197308,0.0
9124,9124,0.686266,0.368073,-0.056639,0.357057,0.499931,0.376432,0.0


In [30]:
from sklearn.metrics import roc_auc_score
roc_auc_score(pd.read_csv("Result/w2v_result_val.csv").iloc[:,-1],pd.read_csv("Result/w2v_result_val.csv").iloc[:,-2])

0.8141005829267601